### Reproducing Figures and Results
This notebook contains the complete statistical analysis pipeline supporting the primary objectives of the paper.  
**_Insights from Reinforcement Learning and individual-based model simulations on population compliance and policy optimization during COVID-19_**


---

###  What’s Included

- Summary statistics for key outcome metrics (mean, standard deviation, standard error, 95% CI)  
- Confidence interval and error bar visualizations  
- Stratified comparisons across cities and intervention models  


---

###  Required Dataset

This notebook relies on the dataset file:

experiment1_results_UP.csv

which located in the project folder:

Master_SiRL_Coderona/data_inputs/experiment1_results_UP.csv

---

###  How to Load the Dataset in Google Colab

1. **Mount your Google Drive:**

```python
from google.colab import drive
drive.mount('/content/drive')

	2.	Load the dataset:

import pandas as pd

# === Dataset Loading ===
data = pd.read_csv("/content/drive/Name_Your_Drive/path_to_the_data_file, header=1)

Replace "Name_Your_Drive/path_to_the_data_file" with the full path to experiment1_results_UP.csv inside your own Google Drive.

 Make sure that the file experiment1_results_UP.csv exists at the exact path specified above.




In [ ]:
# === Import Required Libraries ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive

# === Mount Google Drive ===
# Allows access to the dataset stored in your Google Drive.
drive.mount('/content/drive')

# === Load Dataset ===
# Load simulation results from the specified CSV file and clean column headers.
data = pd.read_csv("/content/drive/Name_Your_Drive/path_to_the_data_file", header=1)
df.columns = df.columns.str.strip()
df['Economic'] = df['Economic'] / 1e9  # Convert economic values to billions (for clarity)

# === Preview Data Structure ===
print("Cities:", df['City'].unique())
print("Models:", df['Model'].unique())

# === Define Parameters ===
# Ordering of intervention strategies and cities
method_order = ['No Intervention', 'Lockdown & Vaccination', 'RL']
city_order = ['Holon', 'Bene Beraq']
metrics = ['Hospitalize', 'Deceased', 'Economic', 'Rt']

# === Summary Statistics ===
n = 30  # Assumed sample size per group
t_value = 2.045  # t-distribution critical value for 95% CI, df = 29

# Group data by city and model, calculate mean and std deviation
summary = df.groupby(['City', 'Model'])[metrics].agg(['mean', 'std'])

# Compute Standard Error (SE) and 95% Confidence Interval (CI) for each metric
for metric in metrics:
    summary[(metric, 'SE')] = summary[(metric, 'std')] / np.sqrt(n)
    summary[(metric, 'CI_95')] = t_value * summary[(metric, 'SE')]

# === Flatten MultiIndex Table ===
flat_summary = summary.stack(level=0).unstack().round(3)
display(flat_summary)  # For Jupyter/Colab
print("\n=== Summary Statistics by City and Model ===")
print(flat_summary.to_string())

# === Metric Extraction Function ===
def extract_metric(city, metric):
    means, stds = [], []
    for method in method_order:
        means.append(summary.loc[(city, method)][(metric, 'mean')])
        stds.append(summary.loc[(city, method)][(metric, 'std')])
    means = np.array(means)
    stds = np.array(stds)
    se = stds / np.sqrt(n)
    ci = t_value * se
    return means, se, ci

# === Plotting ===
fig, axes = plt.subplots(2, 2, figsize=(18, 11))
fig.suptitle(
    'Comparison of Intervention Strategies\nwith 95% Confidence Intervals and Standard Errors',
    fontsize=20, fontweight='bold'
)

titles = ['Cumulative Hospitalized',
          'Cumulative Deceased',
          'Cumulative Economic (Billion USD)',
          'Rt (Reproduction Number)']

colors = {'Holon': "#66b3ff", 'Bene Beraq': "#999999"}
panel_labels = ['(A)', '(B)', '(C)', '(D)']
bar_width = 0.35

# Plot each metric in a separate subplot
for i, metric in enumerate(metrics):
    ax = axes[i // 2, i % 2]
    x = np.arange(len(method_order))

    for j, city in enumerate(city_order):
        means, se, ci = extract_metric(city, metric)
        offset = (j - 0.5) * bar_width  # Offset bars for side-by-side comparison

        # Plot bars with CI
        ax.bar(x + offset, means, width=bar_width, yerr=ci, capsize=5,
               label=f'{city} (95% CI)', color=colors[city], alpha=0.8)

        # Overlay SE
        ax.errorbar(x + offset, means, yerr=se, fmt='none', ecolor='red',
                    elinewidth=1.5, capsize=3, alpha=0.7,
                    label=f'{city} (SE)' if i == 0 else "")

    # Add red line for Rt = 1 threshold
    if metric == 'Rt':
        ax.axhline(y=1.0, color='red', linestyle='--', linewidth=1.2)
        ax.text(len(method_order) - 0.5, 1.02, "R₀ threshold",
                color='red', fontsize=13, ha='left', va='bottom', fontstyle='italic')

    # Title and axes formatting
    ax.set_title(f"{panel_labels[i]} {titles[i]}", fontsize=18, fontweight='bold', loc='left')
    ax.set_xticks(x)
    xtick_labels = ['No Intervention', 'Lockdown & Vaccination', 'Reinforcement Learning']
    ax.set_xticklabels(xtick_labels, fontsize=16)
    ax.tick_params(axis='x', labelrotation=15)
    ax.set_xlabel("Strategy", fontsize=16, fontweight='bold')
    ax.set_ylabel(titles[i], fontsize=16, fontweight='bold')
    ax.tick_params(axis='both', labelsize=14)
    ax.grid(True, linestyle='--', alpha=0.3)

    if i == 0:
        ax.legend(loc='upper right', bbox_to_anchor=(1.0, 1), fontsize=13)

# Adjust layout to avoid overlap
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()